# Unsupervised feature learning

In this problem you will see how unsupervised learning can help you train better models even with labeled data.

In [ ]:
import numpy as np

In [ ]:
image_x = 32 # width of image
image_y = 32 # height of image
patch_dim = 8 # height/width of a patch

## Setting up the environment

In [ ]:
class Image(object):

    def __init__(self,data,label,patches):
        """
        Constructor
        -----------
        Takes image related data, called on image creation.
        """
        self.label = label # image label
        self.patches = patches.transpose().tolist()
        
        self.__img_data = data

    def view(self):
        """
        Function: View
        --------------
        Call function to view RGB image
        """
        from PIL import Image
        im = Image.fromarray(self.__img_data)
        im = im.resize((128,128),Image.BILINEAR)
        im.show()

    def get_label(self):
        """
        Function: Label
        ---------------
        Returns label of image
        """
        return self.label

    def get_patches(self):
        """
        Function: Patches
        -----------------
        Returns list of patch vectors. Each patch length patch_size
        """
        return self.patches

In [ ]:
def load_helper(name,m):
    channels = 3
    patch_dim = 8
    patches_per_image = (image_x/patch_dim)*(image_y/patch_dim)

    images = np.fromfile('data/images_'+name+'.bin',dtype=np.uint8)
    images = images.reshape((m,image_x,image_y,channels))

    patches = np.fromfile('data/patches_'+name+'.bin',dtype=np.float32)
    patches = patches.reshape((patch_dim**2,-1))

    labels = np.fromfile('data/labels_'+name+'.bin',dtype=np.uint8)

    image_list = []
    for i in range(images.shape[0]):
        image_list.append(Image(images[i,...],labels[i],
          patches[:,int(i*patches_per_image):int((i+1)*patches_per_image)]))
    
    return image_list

In [ ]:
def view_helper(patches,num):
    from PIL import Image
    
    xnum = int(np.sqrt(num))
    if xnum**2 == num:
        ynum = xnum
    else:
        ynum = xnum+1

    imDim = 50

    try:
        import matplotlib.pyplot as plt
        patches = patches-np.min(patches)
        patches = patches/np.max(patches)
        patchDim = patches.shape[0]
        image = np.zeros(((patchDim+1)*ynum+1,(patchDim+1)*xnum+1))
        for i in range(ynum):
            for j in range(xnum):
                imnum = i*xnum+j
                if imnum>=num:
                    break
                ax = plt.subplot2grid((ynum,xnum),(i,j))
                ax.imshow(patches[:,:,i*xnum+j].squeeze(), cmap = plt.get_cmap('gray'))
                ax.axes.get_xaxis().set_visible(False)
                ax.axes.get_yaxis().set_visible(False)
                
        plt.subplots_adjust(wspace=-.5 ,hspace=0.2)
        plt.show()
        return
    except ImportError:
        pass
    
    # rescale to be [0-255]
    patches = patches-np.min(patches)
    patches = 255*patches/np.max(patches)

    newpatches = np.empty((imDim,imDim,num))

    for p in range(num):
        patch = patches[:,:,p].squeeze().copy()
        im = Image.fromarray(patch)
        im = im.resize((imDim,imDim),Image.BILINEAR)
        newpatches[:,:,p] = np.asarray(im.convert('L'))

    patches = newpatches
    image = np.zeros(((imDim+1)*ynum+1,(imDim+1)*xnum+1))

    for i in range(ynum):
        for j in range(xnum):
            imnum = i*xnum+j
            if imnum>=num:
                break
            image[i*(imDim+1)+1:i*(imDim+1)+imDim+1, \
                  j*(imDim+1)+1:j*(imDim+1)+imDim+1] \
                  = patches[:,:,imnum]
    image = Image.fromarray(image, 'L')
    image.show()

In [ ]:
def pixel_features(images):
    """
    Extracts raw pixel features for all images.  Returns a 2-D array
    of size featDim x numExamples and a vector of labels.
    """
    X = [np.array(image.get_patches()).ravel() for image in images]
    X = np.vstack(X).transpose() # featdim by num samples
    # label array
    Y = np.array([image.get_label() for image in images])
    return X,Y

In [ ]:
def view_patches(patches):
    """
    Function: View Patches
    ----------------------
    Pass in an array of patches (or centroids) in order to view them as
    images.
    """
    view_helper(patches.reshape(patch_dim,patch_dim,-1),patches.shape[-1])

## Loading the data

In [ ]:
num_train_images = 2000
file_tag = 'train'
train_image_list = load_helper(file_tag,num_train_images)

num_test_images = 1000
file_tag = 'test'
test_image_list = load_helper(file_tag,num_test_images)

In [ ]:
train_X,train_Y = pixel_features(train_image_list)
test_X,test_Y = pixel_features(test_image_list)

In [ ]:
train_X.shape, train_Y.shape

In [ ]:
print(train_X[:10])
print("")
print(train_Y[:10])

## Training logistic regression

Train logistic regression on the raw pixel data and report the train and test set results.

## Training SVM

Train SVM on the raw pixel data and report the train and test set results.

## Training XGBoost

Train XGBoost on the raw pixel data and report the train and test set results.

## Learning better features

Instead of hand-designing better features let us see if we can learn them directly from data. Each image is a 32x32 grid of pixels. We will divide the image into sixteen 8x8 "patches". Next, we will use K-means to cluster all the patches into centroids. These centroids will then allow us to use a better feature representation of the image.

Let us see how we can get patches from the images and visualize them. Make sure you understand the dimensions of every array and what they mean.

In [ ]:
num_images = 2
patches = np.hstack([np.array(image.get_patches()).transpose() for image in image_list[:num_images]])

In [ ]:
patches.shape

In [ ]:
viewPatches(patches)

Run k-means from scikit-learn to group all patches into clusters. Initially, pick the number of clusters according to your best guess.

In [ ]:
num_images = 1000
patches = np.hstack([np.array(image.get_patches()).transpose() for image in image_list[:num_images]])
patches.shape

Now, visualize the centroids.

In [ ]:
centroids = np.array([]) # Please use this variable name for the array of centroids
view_patches(centroids)

## Representing examples in a new way

Now, you have the centroids defining similar groups in your patches. Represent every image in your training and test set in distances between the patch and each centroid. For example, if you used 10 clusters and each image has 16 patches, new representation of the image will be a vector of 160 elements.

## Training classifiers

Train all three classifiers from the above (logistic regression, SVM and XGBoost) on the new image representation. Report the train and test set results.

## Getting the best out of it

In industry, we typically try to get as much as possible out of the data we have. Try different number of clusters and different configuration of the models and report the best accuracy you got on the test set.